# Flood ConvLSTM training

In [1]:
# Setup with GPU
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%load_ext autoreload
%autoreload 2
import tensorflow as tf
tf.config.list_physical_devices('GPU')

env: TF_FORCE_GPU_ALLOW_GROWTH=true


2024-07-15 21:49:21.615643: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 21:49:23.776990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2024-07-15 21:49:23.788029: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 21:49:23.791301: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [2]:
# Load the dataset form firestore
from usl_models.flood_ml.dataset import load_dataset_windowed
from google.cloud import firestore
from google.cloud import storage

sim_names = ["Manhattan-config_v1/Rainfall_Data_1.txt", "Manhattan-config_v1/Rainfall_Data_2.txt"]
batch_size = 1 # On A100, use batch_size = 4
dataset = load_dataset_windowed(sim_names, batch_size=batch_size, max_chunks=8,
                                firestore_client=firestore.Client(project='climateiq-test'),
                                storage_client=storage.Client(project='climateiq-test'))


2024-07-15 21:49:23.976827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 21:49:23.980312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 21:49:23.983369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Initialize the model
from usl_models.flood_ml.model import FloodModel
from usl_models.flood_ml.model_params import FloodModelParams
import keras
model = FloodModel(
    model_params=FloodModelParams(batch_size=batch_size))

In [4]:
# Fit the model to the dataset
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs',
                                         histogram_freq=1,
                                         profile_batch='1,100')]
model.fit(dataset, epochs=1, callbacks=callbacks)

2024-07-15 21:49:24.714333: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-15 21:49:24.714361: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-15 21:49:24.714387: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1239] Profiler found 1 GPUs
2024-07-15 21:49:24.736234: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-15 21:49:24.736359: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed
2024-07-15 21:49:24.764422: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-15 21:49:24.764451: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
I0000 00:00:1721080170.611964   84718 service.cc:145] XLA service 0x7f271c0013a0 initialized for platform CUDA (this does not guarantee that XLA will be used

     99/Unknown 62s 453ms/step - loss: 0.0142 - mean_absolute_error: 0.0251 - root_mean_squared_error: 0.1153

2024-07-15 21:50:27.512033: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-07-15 21:50:27.523707: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed
2024-07-15 21:50:27.587505: I external/local_xla/xla/backends/profiler/gpu/cupti_collector.cc:540]  GpuTracer has collected 164959 callback api events and 160916 activity events. 
2024-07-15 21:50:28.810483: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-15 21:50:28.812051: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: ./logs/plugins/profile/2024_07_15_21_50_28/dl-m123-a100.xplane.pb


    304/Unknown 150s 438ms/step - loss: 0.0105 - mean_absolute_error: 0.0163 - root_mean_squared_error: 0.1002

2024-07-15 21:51:55.043121: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-15 21:51:55.043172: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12677227185886623058
2024-07-15 21:51:55.043183: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-07-15 21:51:55.043236: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13934380519738650739
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, valu

304/304 ━━━━━━━━━━━━━━━━━━━━ 151s 439ms/step - loss: 0.0105 - mean_absolute_error: 0.0162 - root_mean_squared_error: 0.1001


In [5]:
dataset = load_dataset_windowed(sim_names, batch_size=1, max_chunks=8)
input, labels = next(iter(dataset))
prediction = model.call(input)
prediction.shape, tf.math.reduce_max(prediction)

(TensorShape([1, 1000, 1000, 1]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.43272135>)

In [6]:
model.save_model('data/flood_model_tf213_20240711-190530.keras')

[TIP] Next time specify overwrite=True!


In [7]:
# Test call_n
from usl_models.flood_ml.dataset import load_dataset
dataset = load_dataset(sim_names, batch_size=1, max_chunks=1)
input, labels = next(iter(dataset))

(B, n, H, W) = labels.shape
predictions = model.call_n(input, n=n)
predictions.shape, tf.math.reduce_max(predictions)

(TensorShape([1, 19, 1000, 1000]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.01565428>)